In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    if ds.Modality != "DX":
        print("Warning: Image modality is CT")
        img = 0
    
    if ds.BodyPartExamined != "CHEST":
        print("Warning: Body part analized is {}" .format(ds.BodyPartExamined))
        img = 0
    
    if not (ds.PatientPosition == "AP" or ds.PatientPosition == "PA"):
        print("Warning: Patient position is {}" .format(ds.PatientPosition))
        img = 0
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = img.copy()
    proc_img = (proc_img - img_mean)/img_std
    # Finally resizing it to our desired size.
    proc_img = resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    
    json_file = open(model_path, "r")
    loaded_model_json = json_file.read() 
    json_file.close() 
    model = keras.models.model_from_json(loaded_model_json) 
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    
    prediction = my_model.predict(img)
    
    return prediction 

In [10]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path =  "my_model.json" #path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class')#path to saved best weights


IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16


my_model = load_model(model_path, weight_path) #loads model
thresh = 0.50 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is not 0:
        img_mean = np.mean(img) # loads the mean image value they used during training preprocessing
        img_std = np.std(img) # loads the std dev image value they used during training preprocessing
        img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
        pred = predict_image(my_model,img_proc,thresh)
        print(pred, np.where(np.round(pred) == 0, "No pneumonia", "Pneumonia"))
        pass

Load file test1.dcm ...
[[0.5045222]] [['Pneumonia']]
Load file test2.dcm ...
[[0.50220346]] [['Pneumonia']]
Load file test3.dcm ...
[[0.5075497]] [['Pneumonia']]
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...


In [ ]:
# Conclusions:
# 1. Algorithm performance needs to be improved